In [ ]:
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import random_split


In [ ]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1   = nn.BatchNorm2d(planes)
        self.relu  = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2   = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride !=1 or in_planes != planes * self.expansion:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes * self.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * self.expansion)
            )

    def forward(self, x):
        identity = x
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(identity)
        out = self.relu(out)
        return out


In [ ]:
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=100):  # 设置为100类
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1  = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1    = nn.BatchNorm2d(64)
        self.relu   = nn.ReLU(inplace=True)
        self.layer1 = self._make_layer(block, 64,  num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.avgpool= nn.AdaptiveAvgPool2d((1,1))
        self.fc     = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks - 1)
        layers  = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        out = self.fc(out)
        return out  # 返回最终输出


In [ ]:
def ResNet18():
    return ResNet(BasicBlock, [2,2,2,2])


In [ ]:
coarse_label_names = [
    'aquatic mammals', 'fish', 'flowers', 'food containers', 'fruit and vegetables',
    'household electrical devices', 'household furniture', 'insects', 'large carnivores',
    'largxe man-made outdoor things', 'large natural outdoor scenes', 'large omnivores and herbivores',
    'medium-sized mammals', 'non-insect invertebrates', 'people', 'reptiles', 'small mammals',
    'trees', 'vehicles 1', 'vehicles 2'
]


In [ ]:
fine_label_names = [
    'apple', 'aquarium_fish', 'baby', 'bear', 'beaver',
    'bed', 'bee', 'beetle', 'bicycle', 'bottle',
    'bowl', 'boy', 'bridge', 'bus', 'butterfly',
    'camel', 'can', 'castle', 'caterpillar', 'cattle',
    'chair', 'chimpanzee', 'clock', 'cloud', 'cockroach',
    'couch', 'crab', 'crocodile', 'cup', 'dinosaur',
    'dolphin', 'elephant', 'flatfish', 'forest', 'fox',
    'girl', 'hamster', 'house', 'kangaroo', 'keyboard',
    'lamp', 'lawn_mower', 'leopard', 'lion', 'lizard',
    'lobster', 'man', 'maple_tree', 'motorcycle', 'mountain',
    'mouse', 'mushroom', 'oak_tree', 'orange', 'orchid',
    'otter', 'palm_tree', 'pear', 'pickup_truck', 'pine_tree',
    'plain', 'plate', 'poppy', 'porcupine', 'possum',
    'rabbit', 'raccoon', 'ray', 'road', 'rocket',
    'rose', 'sea', 'seal', 'shark', 'shrew',
    'skunk', 'skyscraper', 'snail', 'snake', 'spider',
    'squirrel', 'streetcar', 'sunflower', 'sweet_pepper', 'table',
    'tank', 'telephone', 'television', 'tiger', 'tractor',
    'train', 'trout', 'tulip', 'turtle', 'wardrobe',
    'whale', 'willow_tree', 'wolf', 'woman', 'worm'
]


In [ ]:
# 创建细粒度标签到粗粒度标签的映射
# CIFAR-100数据集的每个超级类别包含5个细粒度类别，按顺序排列
fine_to_coarse = {}
for coarse_label in range(20):
    for i in range(5):
        fine_label = coarse_label * 5 + i
        fine_to_coarse[fine_label] = coarse_label


In [ ]:
# 打印超级类别1和5的信息
print("Superclass 1:", coarse_label_names[1])
print("Superclass 5:", coarse_label_names[5])


In [ ]:
# 数据预处理
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408),
                         (0.2675, 0.2565, 0.2761)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408),
                         (0.2675, 0.2565, 0.2761)),
])


In [ ]:
# 加载完整的训练数据集
full_trainset = torchvision.datasets.CIFAR100(
    root='./data', train=True, download=True, transform=transform_train)


In [ ]:
# 定义训练集和验证集的大小
train_size = int(0.9 * len(full_trainset))
val_size = len(full_trainset) - train_size

# 设置随机种子以确保可重复性
torch.manual_seed(42)
trainset, valset = random_split(full_trainset, [train_size, val_size])


In [ ]:
# 创建数据加载器
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=64, shuffle=True, num_workers=2)
valloader = torch.utils.data.DataLoader(
    valset, batch_size=64, shuffle=False, num_workers=2)

# 加载测试数据集
testset = torchvision.datasets.CIFAR100(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=64, shuffle=False, num_workers=2)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
model = ResNet18().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1,
                      momentum=0.9, weight_decay=5e-4)


In [ ]:
# 学习率调度器
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)

In [ ]:
# 训练模型并记录准确率
num_epochs = 50

train_acc_list = []
val_acc_list = []

best_val_acc = 0  # 用于保存最佳模型


In [ ]:
for epoch in range(num_epochs):
    # 训练阶段
    model.train()
    running_loss = 0.0
    total = 0
    correct = 0
    for batch_idx, (inputs, labels) in enumerate(trainloader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = outputs.topk(5, 1, True, True)
        total += labels.size(0)
        correct += (predicted[:, 0] == labels).sum().item()

    train_acc = 100. * correct / total
    train_acc_list.append(train_acc)
    print(
        f'Epoch [{epoch+1}/{num_epochs}] completed. Training Loss: {running_loss/len(trainloader):.4f}, Training Top-1 Acc: {train_acc:.2f}%')

    # 验证阶段
    model.eval()
    val_loss = 0
    correct_top1 = 0
    correct_top5 = 0
    total = 0

    # 用于计算超级类别的准确率
    correct_superclass = 0
    total_superclass = 0

    with torch.no_grad():
        for inputs, labels in valloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = outputs.topk(5, 1, True, True)
            total += labels.size(0)
            correct_top1 += (predicted[:, 0] == labels).sum().item()
            correct_top5 += sum([labels[i].item() in predicted[i].cpu().numpy()
                                 for i in range(labels.size(0))])

            # 超级类别准确率计算
            labels_coarse = torch.tensor(
                [fine_to_coarse[label.item()] for label in labels], device=device)
            predicted_coarse = torch.tensor(
                [fine_to_coarse[pred.item()] for pred in predicted[:, 0]], device=device)
            correct_superclass += (
                predicted_coarse == labels_coarse).sum().item()
            total_superclass += labels.size(0)

    val_acc_top1 = 100. * correct_top1 / total
    val_acc_top5 = 100. * correct_top5 / total
    val_acc_superclass = 100. * correct_superclass / total_superclass
    val_acc_list.append(val_acc_top1)

    print(f'Validation Loss: {val_loss/len(valloader):.4f}')
    print(f'Top-1 Validation Accuracy: {val_acc_top1:.2f}%')
    print(f'Top-5 Validation Accuracy: {val_acc_top5:.2f}%')
    print(f'Top-1 Superclass Validation Accuracy: {val_acc_superclass:.2f}%')

    # 保存最佳模型
    if val_acc_top1 > best_val_acc:
        best_val_acc = val_acc_top1
        torch.save(model.state_dict(), 'best_model.pth')

    # 更新学习率
    scheduler.step()


In [ ]:
# 绘制训练和验证准确率曲线
plt.figure(figsize=(10, 5))
plt.plot(range(1, num_epochs+1), train_acc_list,
         label='Train Top-1 Accuracy')
plt.plot(range(1, num_epochs+1), val_acc_list,
         label='Validation Top-1 Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.title('Training and Validation Top-1 Accuracy over Epochs')
plt.legend()
plt.show()


In [ ]:
# 在测试集上进行最终评估
model.load_state_dict(torch.load('best_model.pth'))


In [ ]:
model.eval()
test_loss = 0
correct_top1 = 0
correct_top5 = 0
total = 0

# 用于计算超级类别的准确率
correct_superclass = 0
total_superclass = 0

with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        test_loss += loss.item()
        _, predicted = outputs.topk(5, 1, True, True)
        total += labels.size(0)
        correct_top1 += (predicted[:, 0] == labels).sum().item()
        correct_top5 += sum([labels[i].item() in predicted[i].cpu().numpy()
                             for i in range(labels.size(0))])

        # 超级类别准确率计算
        labels_coarse = torch.tensor(
            [fine_to_coarse[label.item()] for label in labels], device=device)
        predicted_coarse = torch.tensor(
            [fine_to_coarse[pred.item()] for pred in predicted[:, 0]], device=device)
        correct_superclass += (
            predicted_coarse == labels_coarse).sum().item()
        total_superclass += labels.size(0)

test_acc_top1 = 100. * correct_top1 / total
test_acc_top5 = 100. * correct_top5 / total
test_acc_superclass = 100. * correct_superclass / total_superclass

print('Final Test Results:')
print(f'Test Loss: {test_loss/len(testloader):.4f}')
print(f'Top-1 Test Accuracy: {test_acc_top1:.2f}%')
print(f'Top-5 Test Accuracy: {test_acc_top5:.2f}%')
print(f'Top-1 Superclass Test Accuracy: {test_acc_superclass:.2f}%')
